In [25]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [26]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-12-13 00:30:03--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  44.4MB/s    in 0.6s    

2024-12-13 00:30:04 (44.4 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [27]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [28]:
# add your code here - consider creating a new cell for each section of code

ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', encoding='latin-1')
books = pd.read_csv('BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')
users = pd.read_csv('BX-Users.csv', sep=';', encoding='latin-1')

# Data cleaning and preprocessing
ratings.columns = ['user', 'ISBN', 'rating']
books.rename(columns={'Book-Title': 'title', 'Book-Author': 'author'}, inplace=True)
users.columns = ['user', 'location', 'age']

ratings = ratings[ratings['rating'] != 0]
ratings_count = pd.DataFrame(ratings.groupby('ISBN')['rating'].count())
ratings_count = ratings_count.sort_values('rating', ascending=False)
ratings_count = ratings_count[ratings_count['rating'] >= 100]

ratings = ratings[ratings['ISBN'].isin(ratings_count.index)]
ratings_count = pd.DataFrame(ratings.groupby('user')['rating'].count())
ratings_count = ratings_count.sort_values('rating', ascending=False)
ratings_count = ratings_count[ratings_count['rating'] >= 200]
ratings = ratings[ratings['user'].isin(ratings_count.index)]

# Create a user-item matrix
ratings_pivot = ratings.pivot(index='user', columns='ISBN', values='rating').fillna(0)

# Function to get book recommendations
def get_recommends(book_title):
    ISBN = books_df[books_df['title'] == book_title]['ISBN'].iloc[0]
    try:
        book_index = ratings_pivot.columns.get_loc(ISBN)
    except KeyError:
        return [book_title, []] # Handle cases where the book is not found

    model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
    model_knn.fit(ratings_pivot.T)
    distances, indices = model_knn.kneighbors(ratings_pivot.T.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)

    recommended_books = []
    for i in range(1, len(distances.flatten())):
        index = indices.flatten()[i]
        ISBN = ratings_pivot.columns[index]
        distance = distances.flatten()[i]
        title = books[books['ISBN'] == ISBN]['title'].iloc[0]
        recommended_books.append([title, distance])

    return [book_title, recommended_books]

<ipython-input-28-d0e4c2ab7d33>:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')


In [29]:
# function to return recommended books - this will be tested
def get_recommends(book_title=""):
  """
  This function takes a book title as input and returns a list of recommended books.

  Args:
    book_title (str): The title of the book to get recommendations for.

  Returns:
    list: A list of recommended books, with each recommendation being a list of [title, distance].
  """
  # Get the ISBN of the input book
  ISBN = books[books['title'] == book_title]['ISBN'].values[0]

  # Handle cases where the book is not found
  try:
    book_index = ratings_pivot.columns.get_loc(ISBN)
  except KeyError:
    return [book_title, []]

  # Create and fit the KNN model
  model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
  model_knn.fit(ratings_pivot.T)

  # Get the nearest neighbors
  distances, indices = model_knn.kneighbors(ratings_pivot.T.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)

  # Build the list of recommended books
  recommended_books = []
  for i in range(1, len(distances.flatten())):
    index = indices.flatten()[i]
    ISBN = ratings_pivot.columns[index]
    distance = distances.flatten()[i]
    title = books[books['ISBN'] == ISBN]['title'].iloc[0]
    recommended_books.append([title, distance])

  # Return the results
  return [book_title, recommended_books]

In [32]:
books_recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books_recommends)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    for i in range(len(recommends[1])):
      if recommends[1][i][0] not in recommended_books:
        test_pass = False
      if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
        test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", []]
You passed the challenge! 🎉🎉🎉🎉🎉
